Let's design a simple diverging lens using two concave surfaces, aka a biconcave lens.

The only difference with a converging lens is actually that the focal length is negative. Is is behind the lens in the sense that rays don't actually converge on the focal point, but they spread out and appear to come from the focal point. If you extend the outgoing rays in the negative direction, they will cross at the focal point.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchlensmaker as tlm


class Optics(tlm.Module):
    def __init__(self):
        super().__init__()

        # A simple parabolic shape for both surfaces
        # Wrap it in a nn.Parameter to enable optimization
        self.shape = tlm.Parabola(width=20., a=nn.Parameter(torch.tensor(0.005)))

        # Symmetric lens sharing the same mirrored shape for both surfaces
        # We set the inner thickness because we know that the lens will be biconcave
        # and therefore the outer thickness will be greater
        self.lens = tlm.SymmetricLens(self.shape, (1.0, 1.49), inner_thickness=1.0)

        # Setup the optical stack with a simple parallel light source and negative focal point
        self.optics = nn.Sequential(
            tlm.PointSourceAtInfinity(beam_diameter=15),
            tlm.Gap(10.), 
            self.lens,
            tlm.Gap(-25.0),
            tlm.FocalPointLoss(),
        )

    def forward(self, inputs):
        return self.optics(inputs)

# Instanciate the optical stack
optics = Optics()

# Render it using the matplotlib renderer
tlm.render_plt(optics)

# Optimize the parameters, here only the shape's parabolic coefficient
tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-3),
    num_iter = 100
)

# Render again after optimization
tlm.render_plt(optics)

In [ ]:
from IPython.display import display
import build123d as bd

part = tlm.lens_to_part(optics.lens)
display(part)

print("Outer thickness:", optics.lens.outer_thickness().item())
print("Inner thickness:", optics.lens.inner_thickness().item())

# bd.export_step(part, "lens.step")